# Make-a-bot
Baixe os tweets da sua personalidade favorita e faça uma RNN que tenta prever as próximas palavras.

In [ ]:
#!conda install -v -c conda-forge tweepy
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.text import *
from pathlib import Path

from tweet_dumper import get_all_tweets

In [ ]:
tweetpath = Path('./tweets')
modelspath = Path('./models')
path = Path('./')
best_model_path = Path('./models/bestmodel30k')

In [ ]:
# Download do modelo em portugues pretreinado na wikipedia
!curl https://storage.googleapis.com/gde-dl-bsb/models/bestmodel30k.pth -o ../models/bestmodel30k.pth
# Download do vocabulário
!curl https://storage.googleapis.com/gde-dl-bsb/models/itos.pkl -o ../models/itos.pkl

## Baixando os tweets

Escreva suas chaves de acesso da API do Twitter e defina o nome do perfil de twitter que gostaria de usar para treinar a rede.

Mais informações em https://developer.twitter.com/

In [ ]:
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""

keys = [consumer_key,consumer_secret,access_key,access_secret]

In [ ]:
screen_name = "jairbolsonaro"

In [ ]:
get_all_tweets(screen_name, keys)

## Preparando os dados

In [ ]:
def readtweets(d): return [' '.join(o.strip() for o in open(d).readlines())]
tweets = 'tweets/' + '{}_tweets.txt'.format(screen_name)
t = readtweets(tweets)

#separa 80% dos tweets para treino e 20% para validação
perc80 = len(t) - len(t)//5

train_txt = valid_txt = []
train_txt.append(t[0][:perc80])
valid_txt.append(t[0][perc80:])

In [ ]:
bs = 64

train = TextList(list(train_txt), path=path);
valid = TextList(list(valid_txt), path=path);

src = ItemLists(path=path, train=train, valid=valid).label_for_lm()
data = src.databunch(bs=bs)

In [ ]:
#tokenizador
tokenizer = Tokenizer(lang='pt', n_cpus=8)
#vocabulario
with modelspath.joinpath('itos.pkl').open('rb') as f:
    itos = pickle.load(f)
vocab = Vocab(itos)

## Transfer Learning

In [ ]:
learn = language_model_learner(data, arch=AWD_LSTM, pretrained_fnames=(best_model_path,modelspath.joinpath('itos')))

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, max_lr=5e-2)

In [ ]:
learn.fit_one_cycle(5, max_lr=5e-2)

In [ ]:
learn.save('first-try')

## Fine Tuning

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn.save('fine-tuned-model')

## Prediction

In [ ]:
TEXT = ""
N_WORDS = 50
N_SENTENCES = 5
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

----
# Load Examples

Podemos carregar robôs pré-treinados também. Alguns dos resultados já estão disponíveis na pasta *examples*.

## Bolsobot

In [ ]:
# Download do modelo pré-treinado do Bolsonaro
!curl https://www.dropbox.com/s/bn4i4x6jiinejzy/Bolsobot.pth -o ../models/Bolsobot.pth

In [ ]:
learn.load('Bolsobot')

In [ ]:
TEXT = "Grande dia!"
N_WORDS = 50
N_SENTENCES = 5
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

## Robolavo

In [ ]:
# Download do modelo pré-treinado do Olavo de Carvalho
!curl https://www.dropbox.com/s/yhsbw1yu9a0844x/olavobot-twitter.pth -o ../models/olavobot-twitter.pth

In [ ]:
learn.load('olavobot-twitter')

In [ ]:
TEXT = "A Terra é"
N_WORDS = 50
N_SENTENCES = 5
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))